In [1]:
from sklearn.datasets import load_wine
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

import numpy as np
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import GradientBoostingClassifier as gb


In [12]:
wine = load_wine()
X, Y = shuffle(wine['data'], wine['target'], random_state=1000)


In [13]:
nb_samples = X.shape[0]
nb_labeled = 20
nb_unlabeled = nb_samples - nb_labeled
nb_unlabeled_samples = 2

feature_cut = 7
X_unlabeled = X[-nb_unlabeled:]
X_labeled = X[:nb_labeled]
Y_labeled = Y[:nb_labeled]
X_labeled_1 = X_labeled[:, 0:feature_cut]
X_labeled_2 = X_labeled[:, feature_cut:]

In [5]:
nb0 = GaussianNB()
nb0.fit(X_labeled, Y_labeled)
print(classification_report(Y, nb0.predict(X), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.51      0.67        59
     class_1       0.68      1.00      0.81        71
     class_2       1.00      0.92      0.96        48

    accuracy                           0.81       178
   macro avg       0.89      0.81      0.81       178
weighted avg       0.87      0.81      0.81       178



In [6]:
nb0 = gb()
nb0.fit(X_labeled, Y_labeled)
print(classification_report(Y, nb0.predict(X), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.78      0.88        59
     class_1       0.85      0.99      0.92        71
     class_2       0.94      0.98      0.96        48

    accuracy                           0.92       178
   macro avg       0.93      0.91      0.92       178
weighted avg       0.93      0.92      0.91       178



In [7]:
nb0 = rf()
nb0.fit(X_labeled, Y_labeled)
print(classification_report(Y, nb0.predict(X), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       0.98      0.80      0.88        59
     class_1       0.85      0.99      0.92        71
     class_2       1.00      1.00      1.00        48

    accuracy                           0.93       178
   macro avg       0.94      0.93      0.93       178
weighted avg       0.93      0.93      0.93       178



In [8]:
nb1 = None
nb2 = None
while X_labeled_1.shape[0] <= nb_samples:
    
    nb1 = GaussianNB()
    nb1.fit(X_labeled_1, Y_labeled)
    
    nb2 = gb()
    nb2.fit(X_labeled_2, Y_labeled)
    
    if X_labeled_1.shape[0] == nb_samples:
        break
        
    probs1 = nb1.predict_proba(X_unlabeled[:, 0:feature_cut])
    top_confidence_idxs1 = np.argsort(np.max(probs1, axis=1))[::-1]
    selected_idxs1 = top_confidence_idxs1[0:nb_unlabeled_samples]
    
    probs2 = nb2.predict_proba(X_unlabeled[:, feature_cut:])
    top_confidence_idxs2 = np.argsort(np.max(probs2, axis=1))[::-1]
    selected_idxs2 = top_confidence_idxs2[0:nb_unlabeled_samples]
    
    selected_idxs = list(selected_idxs1) + list(selected_idxs2)
    
    X_new_labeled = X_unlabeled[selected_idxs]
    X_new_labeled_1 = X_unlabeled[selected_idxs1, 0:feature_cut]
    X_new_labeled_2 = X_unlabeled[selected_idxs2, feature_cut:]
    Y_new_labeled_1 = nb1.predict(X_new_labeled_1)
    Y_new_labeled_2 = nb2.predict(X_new_labeled_2)
    
    X_labeled_1 = np.concatenate((X_labeled_1, X_new_labeled[:, 0:feature_cut]), axis=0)
    X_labeled_2 = np.concatenate((X_labeled_2, X_new_labeled[:, feature_cut:]), axis=0)
    Y_labeled = np.concatenate((Y_labeled, Y_new_labeled_1, Y_new_labeled_2), axis=0)
    
    X_unlabeled = np.delete(X_unlabeled, selected_idxs, axis=0)


In [9]:
#gaussianNB
print(classification_report(Y, nb1.predict(X[:, 0:feature_cut]), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.17      0.29        59
     class_1       0.56      0.99      0.71        71
     class_2       0.98      0.88      0.92        48

    accuracy                           0.69       178
   macro avg       0.85      0.68      0.64       178
weighted avg       0.82      0.69      0.63       178



In [10]:
#gradient boosting
print(classification_report(Y, nb2.predict(X[:, feature_cut:]), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.08      0.16        59
     class_1       0.55      1.00      0.71        71
     class_2       1.00      0.90      0.95        48

    accuracy                           0.67       178
   macro avg       0.85      0.66      0.60       178
weighted avg       0.82      0.67      0.59       178



In [14]:
nb1 = None
nb2 = None
while X_labeled_1.shape[0] <= nb_samples:
    
    nb1 = GaussianNB()
    nb1.fit(X_labeled_1, Y_labeled)
    
    nb2 = rf()
    nb2.fit(X_labeled_2, Y_labeled)
    
    if X_labeled_1.shape[0] == nb_samples:
        break
        
    probs1 = nb1.predict_proba(X_unlabeled[:, 0:feature_cut])
    top_confidence_idxs1 = np.argsort(np.max(probs1, axis=1))[::-1]
    selected_idxs1 = top_confidence_idxs1[0:nb_unlabeled_samples]
    
    probs2 = nb2.predict_proba(X_unlabeled[:, feature_cut:])
    top_confidence_idxs2 = np.argsort(np.max(probs2, axis=1))[::-1]
    selected_idxs2 = top_confidence_idxs2[0:nb_unlabeled_samples]
    
    selected_idxs = list(selected_idxs1) + list(selected_idxs2)
    X_new_labeled = X_unlabeled[selected_idxs]
    X_new_labeled_1 = X_unlabeled[selected_idxs1, 0:feature_cut]
    X_new_labeled_2 = X_unlabeled[selected_idxs2, feature_cut:]
    Y_new_labeled_1 = nb1.predict(X_new_labeled_1)
    Y_new_labeled_2 = nb2.predict(X_new_labeled_2)
    
    X_labeled_1 = np.concatenate((X_labeled_1, X_new_labeled[:, 0:feature_cut]), axis=0)
    X_labeled_2 = np.concatenate((X_labeled_2, X_new_labeled[:, feature_cut:]), axis=0)
    Y_labeled = np.concatenate((Y_labeled, Y_new_labeled_1, Y_new_labeled_2), axis=0)
    
    X_unlabeled = np.delete(X_unlabeled, selected_idxs, axis=0)



In [15]:
#gaussian
print(classification_report(Y, nb1.predict(X[:, 0:feature_cut]), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.19      0.31        59
     class_1       0.56      0.99      0.72        71
     class_2       0.98      0.88      0.92        48

    accuracy                           0.69       178
   macro avg       0.85      0.68      0.65       178
weighted avg       0.82      0.69      0.64       178



In [16]:
#rf
print(classification_report(Y, nb2.predict(X[:, feature_cut:]), target_names=wine['target_names']))


              precision    recall  f1-score   support

     class_0       1.00      0.07      0.13        59
     class_1       0.55      1.00      0.71        71
     class_2       1.00      0.92      0.96        48

    accuracy                           0.67       178
   macro avg       0.85      0.66      0.60       178
weighted avg       0.82      0.67      0.58       178

